## Домашнее задание по теме "Функции потерь и оптимизация"

***Цель:*** оптимизировать самостоятельно реализованную логистическую регрессию.   
***Задачи:***
- реализовать логистическую регрессию
- обучить ее методом градиентного спуска
- методом nesterov momentum
- методом rmsprop

***План*** (в основном совпадает с задачами):
1. Изучить данные и предобработать их (проверить на пропуски; оставить только 2 класса - Iris Versicolor и Iris Virginica; проверить баланс классов)  
2. Написать функцию логистической регрессии, вручную подобрав коэффициенты; вызвать функцию, проверить качество обучения.  
3. Написать функцию градиентного спуска, обучить модель этим методом, проверить score.  
4. Написать функцию nesterov momentum, обучить модель этим методом, проверить score.  
5. Написать функцию для метода rmsprop, обучить модель этим методом, проверить score.  
6. Оформить выводы; выбрать наиболее успешный метод оптимизации.

### Шаг 1. Изучение и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

Познакомимся с датасетом "Ирисы":

In [3]:
iris = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

In [4]:
df = pd.read_csv(iris, sep=',')

In [5]:
attributes = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df.columns = attributes

In [6]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  149 non-null    float64
 1   sepal_width   149 non-null    float64
 2   petal_length  149 non-null    float64
 3   petal_width   149 non-null    float64
 4   class         149 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


Проверим, сколько разных классов используется в потенциальном целевом признаке:

In [8]:
len(df['class'].unique())

3

In [9]:
df['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

Согласно условиям задачи, класс 'Iris-setosa' нам не нужен. Удалим строки, где в качестве целевого признака указан этот класс:

In [10]:
len(df[df['class'] == 'Iris-setosa'])

49

In [11]:
df = df[df['class'] != 'Iris-setosa']

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 49 to 148
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  100 non-null    float64
 1   sepal_width   100 non-null    float64
 2   petal_length  100 non-null    float64
 3   petal_width   100 non-null    float64
 4   class         100 non-null    object 
dtypes: float64(4), object(1)
memory usage: 4.7+ KB


In [13]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
49,7.0,3.2,4.7,1.4,Iris-versicolor
50,6.4,3.2,4.5,1.5,Iris-versicolor
51,6.9,3.1,4.9,1.5,Iris-versicolor
52,5.5,2.3,4.0,1.3,Iris-versicolor
53,6.5,2.8,4.6,1.5,Iris-versicolor


In [14]:
df['class'].unique()

array(['Iris-versicolor', 'Iris-virginica'], dtype=object)

В принципе, это не очень важно, но мы можем сбросить индексы:

In [15]:
df = df.reset_index(drop=True)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,7.0,3.2,4.7,1.4,Iris-versicolor
1,6.4,3.2,4.5,1.5,Iris-versicolor
2,6.9,3.1,4.9,1.5,Iris-versicolor
3,5.5,2.3,4.0,1.3,Iris-versicolor
4,6.5,2.8,4.6,1.5,Iris-versicolor


Теперь перекодируем столбец "class" и обозначим его как целевой признак:

In [16]:
lec = LabelEncoder()
lec.fit(df['class'])
target = pd.Series(data=lec.transform(df['class']))
target.head()

0    0
1    0
2    0
3    0
4    0
dtype: int32

In [17]:
target.tail()

95    1
96    1
97    1
98    1
99    1
dtype: int32

Проверим баланс классов в таргете:

In [18]:
target.value_counts()

1    50
0    50
dtype: int64

Баланс идеален. Такое вряд ли встретится в реальной жизни.  
Идем дальше: проверим на наличие пропусков остальные столбцы фрейма и обозначим их как фичи (к слову, с типами данных всё в порядке):

In [19]:
df.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
class           0
dtype: int64

In [20]:
features = df.drop('class', axis=1)
features.head()

,sepal_length,sepal_width,petal_length,petal_width
0,7.0,3.2,4.7,1.4
1,6.4,3.2,4.5,1.5
2,6.9,3.1,4.9,1.5
3,5.5,2.3,4.0,1.3
4,6.5,2.8,4.6,1.5


Разделим выборку на обучающую и тестовую:

In [21]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [22]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(70, 4)
(30, 4)
(70,)
(30,)


Отлично! Данные готовы к обучению. Переходим ко второму шагу - напишем функцию логистической регрессии.

### Шаг 2. Логистическая регрессия

Напишем простую функцию логистической регрессии, принимающую на вход строку фичей и подобранные вручную коэффициенты. Для каждой строки расчитаем по формуле лог регрессии прогнозное значение, которое и вернем:

In [23]:
def log_reg(row, coefficients):
    y_hat = coefficients[0]        # упростим формулу, разбив ее на несколько строк
    for i in range(len(row)):
        y_hat += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + np.exp(-y_hat))   # найдем прогнозное значение, используя производную функции

Рандомным способом выберем 5 коэффициентов для расчета прогноза:

In [24]:
# rand_coef = np.random.default_rng().random(5)
# rand_coef

Из всех вариантов перебора наиболее выигрышными были те, что зафиксированы в ячейке ниже (с отрицательными значениями немного поиграла вручную):

In [25]:
coef = [-0.30362818, -0.80180026, 0.55469071, 0.749991, -0.00459491]
coef

[-0.30362818, -0.80180026, 0.55469071, 0.749991, -0.00459491]

Вызовем функцию логистической регрессии:

In [26]:
pred = []
for row in features_train.to_numpy():
    yhat = log_reg(row, coef)
    print("Predicted=%.3f [%d]" % (yhat, round(yhat)))
    pred.append(round(yhat))

Predicted=0.443 [0]
Predicted=0.390 [0]
Predicted=0.438 [0]
Predicted=0.466 [0]
Predicted=0.607 [1]
Predicted=0.512 [1]
Predicted=0.566 [1]
Predicted=0.575 [1]
Predicted=0.535 [1]
Predicted=0.559 [1]
Predicted=0.437 [0]
Predicted=0.598 [1]
Predicted=0.394 [0]
Predicted=0.469 [0]
Predicted=0.433 [0]
Predicted=0.381 [0]
Predicted=0.565 [1]
Predicted=0.346 [0]
Predicted=0.506 [1]
Predicted=0.483 [0]
Predicted=0.353 [0]
Predicted=0.391 [0]
Predicted=0.405 [0]
Predicted=0.485 [0]
Predicted=0.368 [0]
Predicted=0.576 [1]
Predicted=0.498 [0]
Predicted=0.602 [1]
Predicted=0.392 [0]
Predicted=0.481 [0]
Predicted=0.627 [1]
Predicted=0.610 [1]
Predicted=0.620 [1]
Predicted=0.362 [0]
Predicted=0.724 [1]
Predicted=0.656 [1]
Predicted=0.470 [0]
Predicted=0.532 [1]
Predicted=0.507 [1]
Predicted=0.510 [1]
Predicted=0.483 [0]
Predicted=0.478 [0]
Predicted=0.513 [1]
Predicted=0.318 [0]
Predicted=0.657 [1]
Predicted=0.542 [1]
Predicted=0.547 [1]
Predicted=0.369 [0]
Predicted=0.603 [1]
Predicted=0.619 [1]


Теперь сравним тергеты и предсказанные значения, вычислив долю правильных ответов. Для этого напишем функцию, создающую матрицу правильных и неправильных ответов:

In [27]:
def my_confusion_matrix(y_true, y_pred):
    TP = 0
    FN = 0
    TN = 0
    FP = 0
    for i in range(len(y_true)):
        if (y_true[i] == 1) & (y_pred[i] == 1):
            TP += 1
        elif (y_true[i] == 1) & (y_pred[i] == 0):
            FN += 1
        elif (y_true[i] == 0) & (y_pred[i] == 1):
            FP += 1
        else:
            TN += 1
         
    return np.array([[TN,FP],[FN,TP]])

In [28]:
conf_matr = my_confusion_matrix(target_train.reset_index(drop=True), pred)
conf_matr

array([[28,  5],
       [ 5, 32]])

Перепроверим себя:

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(target_train.reset_index(drop=True), pred)

array([[28,  5],
       [ 5, 32]], dtype=int64)

Наша функция сработала верно. При условии вручную подобранных коэффициентов ошибок первого и второго рода сравнительно немного.  
Теперь всё готово для подсчёта доли правильных ответов - accuracy:

In [30]:
def accuracy(matrix_error):
    true_answers = matrix_error[0][0] + matrix_error[1][1]
    return true_answers / sum(sum(matrix_error))

print("Accuracy:", accuracy(conf_matr))

Accuracy: 0.8571428571428571


Перепроверим себя:

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(target_train, pred)

0.8571428571428571

Да, всё посчитано верно. Таким образом, можно сказать, что на обучающей выборке доля правильных ответов достаточно высока (для ручного перебора весов). Проверим теперь работу модели на тестовой выборке:

In [32]:
pred_test = []
for row in features_test.to_numpy():
    yhat_test = log_reg(row, coef)
    print("Predicted=%.3f [%d]" % (yhat_test, round(yhat_test)))
    pred_test.append(round(yhat_test))

Predicted=0.504 [1]
Predicted=0.610 [1]
Predicted=0.551 [1]
Predicted=0.484 [0]
Predicted=0.462 [0]
Predicted=0.418 [0]
Predicted=0.426 [0]
Predicted=0.471 [0]
Predicted=0.358 [0]
Predicted=0.349 [0]
Predicted=0.335 [0]
Predicted=0.369 [0]
Predicted=0.452 [0]
Predicted=0.550 [1]
Predicted=0.558 [1]
Predicted=0.373 [0]
Predicted=0.468 [0]
Predicted=0.534 [1]
Predicted=0.289 [0]
Predicted=0.352 [0]
Predicted=0.552 [1]
Predicted=0.535 [1]
Predicted=0.352 [0]
Predicted=0.373 [0]
Predicted=0.462 [0]
Predicted=0.340 [0]
Predicted=0.506 [1]
Predicted=0.443 [0]
Predicted=0.486 [0]
Predicted=0.523 [1]


In [33]:
target_test

83    1
53    1
70    1
45    0
44    0
39    0
22    0
80    1
10    0
0     0
18    0
30    0
73    1
33    0
90    1
4     0
76    1
77    1
12    0
31    0
55    1
88    1
26    0
42    0
69    1
15    0
40    0
96    1
9     0
72    1
dtype: int32

На первый взгляд неплохо. Посмотрим на accuracy:

In [34]:
conf_matr_test = my_confusion_matrix(target_test.reset_index(drop=True), pred_test)
print("Матрица ошибок")
print(conf_matr_test)
print("Accuracy_test:", accuracy(conf_matr_test))

Матрица ошибок
[[15  2]
 [ 5  8]]
Accuracy_test: 0.7666666666666667


In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(target_test, pred_test)

0.7666666666666667

И тут результат довольно приличный. Тем не менее, нужно обратить внимание на то, что разница между accuracy на тренировочной и на тестовой выборке достаточно большая. Из этого следует вывод, что модель переобучилась.

И всё же пока с этим ничего делать не будем. Попробуем достичь минимума при помощи градиентного спуска.

### Шаг 3. Градиентный спуск

Итак, уравнение сигмоиды можно представить так:

\begin{align*}
y_p = 1.0 / (1.0 + e^{-(b0 + b1 * X1 + b2 * X2 + b3 * X3 + b4 * X4)}
\end{align*}  

Для каждого объекта, соответственно. Градиентный спуск поможет найти минимум. Задача в том, чтобы найти не просто 5 значений b,
а 5 оптимальных значений для всех параметров х каждого объекта. Т.е. подобрать подходящие веса для модели.  
Для начала переведем наши данные в нужный там тип - array:

In [36]:
f_tr = features_train.to_numpy()
t_tr = target_train.to_numpy()

f_tt = features_test.to_numpy()
t_tt = target_test.to_numpy()

Для того, чтобы в перспективе было удобно определять коэффициент b0, добавим к фичам новый столбец в начале - единицы:

In [37]:
ones = np.ones((f_tr.shape[0], 1))
f_tr = np.hstack([ones, f_tr])
f_tr[:5]

array([[1. , 5.9, 3. , 4.2, 1.5],
       [1. , 6.2, 2.9, 4.3, 1.3],
       [1. , 7.7, 3. , 6.1, 2.3],
       [1. , 6. , 2.9, 4.5, 1.5],
       [1. , 6.8, 3.2, 5.9, 2.3]])

In [38]:
ones_tt = np.ones((f_tt.shape[0], 1))
f_tt = np.hstack([ones_tt, f_tt])
f_tt[:5]

array([[1. , 6.3, 2.8, 5.1, 1.5],
       [1. , 6.3, 2.9, 5.6, 1.8],
       [1. , 6.9, 3.2, 5.7, 2.3],
       [1. , 5.7, 3. , 4.2, 1.2],
       [1. , 5.6, 2.7, 4.2, 1.3]])

Теперь у нас 5 признаков.

Видоизменим функцию расчета предикта:

In [39]:
def predict(coef, train):
    predict = coef[0]* train[:, 0] + coef[1] * train[:, 1] + coef[2] * train[:, 2] + coef[3] * train[:, 3] + coef[4] * train[:, 4]
    sigm = 1. / (1 + np.exp(-predict))
    return sigm

Теперь напишем функцию градиента:

In [40]:
def gradient_descent(start, l_rate, n_epochs, train, y):
    cost = []
    theta = start
    for i in range(n_epochs):
        sigm = predict(theta, train)
        grad = np.matmul(train.T, (sigm - y)) / len(sigm)                # рассчитываем градиент
        theta = theta - l_rate * grad                                    # перезаписываем веса
        loss = - np.mean(np.log(sigm) * y + np.log(1 - sigm) * (1 - y))  # считаем потери
        cost.append(loss)
        i += 1
    return theta, cost

Для старта возьмем вручную подобранные веса из предыдущего шага:

In [41]:
coef_gd, cost_loss = gradient_descent(coef, 0.1, 100, f_tr, t_tr)
coef_gd

array([-0.57813038, -1.30208057, -0.02651767,  1.57869983,  0.72461401])

In [42]:
cost_loss[-10:]

[0.3827931064141419,
 0.3815100429098301,
 0.38023719320412247,
 0.37897444012415443,
 0.3777216681528722,
 0.3764787634029759,
 0.37524561359127206,
 0.3740221080134261,
 0.3728081375191135,
 0.37160359448756053]

Предскажем вероятность попадания в класс 1 (порог поставим 0,5):

In [43]:
pred_gd_train = []
for row in features_train.to_numpy():
    yhat_gd_tr = log_reg(row, coef_gd)
    print("Predicted=%.3f [%d]" % (yhat_gd_tr, round(yhat_gd_tr)))
    pred_gd_train.append(round(yhat_gd_tr))

Predicted=0.349 [0]
Predicted=0.269 [0]
Predicted=0.649 [1]
Predicted=0.431 [0]
Predicted=0.812 [1]
Predicted=0.493 [0]
Predicted=0.704 [1]
Predicted=0.738 [1]
Predicted=0.446 [0]
Predicted=0.560 [1]
Predicted=0.341 [0]
Predicted=0.623 [1]
Predicted=0.252 [0]
Predicted=0.621 [1]
Predicted=0.436 [0]
Predicted=0.272 [0]
Predicted=0.643 [1]
Predicted=0.216 [0]
Predicted=0.667 [1]
Predicted=0.459 [0]
Predicted=0.221 [0]
Predicted=0.367 [0]
Predicted=0.268 [0]
Predicted=0.369 [0]
Predicted=0.260 [0]
Predicted=0.798 [1]
Predicted=0.428 [0]
Predicted=0.830 [1]
Predicted=0.294 [0]
Predicted=0.599 [1]
Predicted=0.788 [1]
Predicted=0.734 [1]
Predicted=0.836 [1]
Predicted=0.229 [0]
Predicted=0.918 [1]
Predicted=0.813 [1]
Predicted=0.377 [0]
Predicted=0.868 [1]
Predicted=0.682 [1]
Predicted=0.631 [1]
Predicted=0.570 [1]
Predicted=0.420 [0]
Predicted=0.749 [1]
Predicted=0.148 [0]
Predicted=0.810 [1]
Predicted=0.748 [1]
Predicted=0.677 [1]
Predicted=0.236 [0]
Predicted=0.805 [1]
Predicted=0.801 [1]


Построим матрицу и посчитаем accuracy:

In [44]:
conf_matr_gd_tr = my_confusion_matrix(t_tr, pred_gd_train)
print("Матрица ошибок")
print(conf_matr_gd_tr)
print("Accuracy_test:", accuracy(conf_matr_gd_tr))

Матрица ошибок
[[30  3]
 [ 0 37]]
Accuracy_test: 0.9571428571428572


Совсем немного ошибок и accuracy высокий. Что скажет тестовая выборка?

In [45]:
pred_gd_test = []
for row in features_test.to_numpy():
    yhat_gd_tt = log_reg(row, coef_gd)
    print("Predicted=%.3f [%d]" % (yhat_gd_tt, round(yhat_gd_tt)))
    pred_gd_test.append(round(yhat_gd_tt))

Predicted=0.570 [1]
Predicted=0.784 [1]
Predicted=0.735 [1]
Predicted=0.359 [0]
Predicted=0.409 [0]
Predicted=0.366 [0]
Predicted=0.494 [0]
Predicted=0.672 [1]
Predicted=0.291 [0]
Predicted=0.207 [0]
Predicted=0.373 [0]
Predicted=0.268 [0]
Predicted=0.547 [1]
Predicted=0.679 [1]
Predicted=0.768 [1]
Predicted=0.317 [0]
Predicted=0.539 [1]
Predicted=0.608 [1]
Predicted=0.196 [0]
Predicted=0.225 [0]
Predicted=0.800 [1]
Predicted=0.602 [1]
Predicted=0.286 [0]
Predicted=0.266 [0]
Predicted=0.630 [1]
Predicted=0.194 [0]
Predicted=0.502 [1]
Predicted=0.604 [1]
Predicted=0.438 [0]
Predicted=0.794 [1]


In [46]:
conf_matr_gd_tt = my_confusion_matrix(target_test.to_numpy(), pred_gd_test)
print("Матрица ошибок")
print(conf_matr_gd_tt)
print("Accuracy_test:", accuracy(conf_matr_gd_tt))

Матрица ошибок
[[15  2]
 [ 0 13]]
Accuracy_test: 0.9333333333333333


На тестовых данных также отличный результат. Т.к. разница между долей верных ответов на тренировочной и тестовой выборках минимальна, можно сделать вывод, что модель не переобучилась.  
Переходим к следующему шагу:

### Шаг 4. Nesterov momentum

Метод "nesterov momentum" не только запоминает скорость на предыдущем шаге и добавляет в α раз меньшую величину на следующем шаге, но и поддерживает идею "заглядывания вперёд". Напишем функцию, оценивающую градиент сразу в позиции последующего шага, с помощью которой можно быстрее и четче найти точку в окрестности, где окажемся в следующий момент. Таким образом, бестрее доберемся до минимума:

In [47]:
def nesterov_momentum(start, l_rate, n_epochs, train, y):
    nm_cost = []
    coef_nm = start
    vel_new = np.zeros(5)
    vel_old = np.zeros(5)
    a = 0.97   # зададим коэффициент а, чтобы использовать как экспоненциально убывающий фактор (уменьшим вес старых значений градиента)
    for i in range(n_epochs):
        sigm = predict(coef_nm, train)
        loss = - np.mean(np.log(sigm) * y + np.log(1 - sigm) * (1 - y))
        nm_cost.append(loss)

        sigm = predict(coef_nm - a * vel_new, train)
        vel_new = (a * vel_new + l_rate * np.matmul((sigm - y), train))/len(sigm)
        coef_nm -= vel_new
        i += 1

        vel_new = vel_old
    return coef_nm, nm_cost

В качестве старта снова возьмем вручную подобранные точки для логистической регрессии в переменной "coef":

In [48]:
coef

[-0.30362818, -0.80180026, 0.55469071, 0.749991, -0.00459491]

In [49]:
nm_coef, cost_nm = nesterov_momentum(coef, 0.02, 800, f_tr, t_tr)
nm_coef

array([-0.70540326, -1.51043065, -0.2927892 ,  1.91673383,  1.0396117 ])

In [50]:
pred_nm_train = []
for row in features_train.to_numpy():
    yhat_nm_tr = log_reg(row, nm_coef)
    print("Predicted=%.3f [%d]" % (yhat_nm_tr, round(yhat_nm_tr)))
    pred_nm_train.append(round(yhat_nm_tr))

Predicted=0.292 [0]
Predicted=0.210 [0]
Predicted=0.705 [1]
Predicted=0.394 [0]
Predicted=0.857 [1]
Predicted=0.461 [0]
Predicted=0.733 [1]
Predicted=0.776 [1]
Predicted=0.383 [0]
Predicted=0.536 [1]
Predicted=0.284 [0]
Predicted=0.609 [1]
Predicted=0.191 [0]
Predicted=0.658 [1]
Predicted=0.413 [0]
Predicted=0.218 [0]
Predicted=0.645 [1]
Predicted=0.159 [0]
Predicted=0.705 [1]
Predicted=0.425 [0]
Predicted=0.164 [0]
Predicted=0.339 [0]
Predicted=0.205 [0]
Predicted=0.303 [0]
Predicted=0.205 [0]
Predicted=0.849 [1]
Predicted=0.374 [0]
Predicted=0.879 [1]
Predicted=0.238 [0]
Predicted=0.622 [1]
Predicted=0.826 [1]
Predicted=0.760 [1]
Predicted=0.881 [1]
Predicted=0.173 [0]
Predicted=0.948 [1]
Predicted=0.843 [1]
Predicted=0.318 [0]
Predicted=0.926 [1]
Predicted=0.725 [1]
Predicted=0.656 [1]
Predicted=0.580 [1]
Predicted=0.372 [0]
Predicted=0.807 [1]
Predicted=0.096 [0]
Predicted=0.842 [1]
Predicted=0.797 [1]
Predicted=0.701 [1]
Predicted=0.177 [0]
Predicted=0.850 [1]
Predicted=0.840 [1]


Построим матрицу и посчитаем accuracy:


In [51]:
conf_matr_nm_tr = my_confusion_matrix(t_tr, pred_nm_train)
print("Матрица ошибок")
print(conf_matr_nm_tr)
print("Accuracy_test:", accuracy(conf_matr_nm_tr))

Матрица ошибок
[[30  3]
 [ 0 37]]
Accuracy_test: 0.9571428571428572


Здесь доля правильных ответов такая же, как при градиентном спуске. Что скажет тестовая выборка?

In [52]:
pred_nm_test = []
for row in features_test.to_numpy():
    yhat_nm_tt = log_reg(row, nm_coef)
    print("Predicted=%.3f [%d]" % (yhat_nm_tt, round(yhat_nm_tt)))
    pred_nm_test.append(round(yhat_nm_tt))

Predicted=0.573 [1]
Predicted=0.823 [1]
Predicted=0.778 [1]
Predicted=0.290 [0]
Predicted=0.365 [0]
Predicted=0.326 [0]
Predicted=0.500 [0]
Predicted=0.724 [1]
Predicted=0.251 [0]
Predicted=0.148 [0]
Predicted=0.371 [0]
Predicted=0.216 [0]
Predicted=0.563 [1]
Predicted=0.707 [1]
Predicted=0.817 [1]
Predicted=0.276 [0]
Predicted=0.545 [1]
Predicted=0.614 [1]
Predicted=0.154 [0]
Predicted=0.170 [0]
Predicted=0.855 [1]
Predicted=0.605 [1]
Predicted=0.242 [0]
Predicted=0.212 [0]
Predicted=0.675 [1]
Predicted=0.137 [0]
Predicted=0.477 [0]
Predicted=0.647 [1]
Predicted=0.397 [0]
Predicted=0.854 [1]


In [53]:
conf_matr_nm_tt = my_confusion_matrix(t_tt, pred_nm_test)
print("Матрица ошибок")
print(conf_matr_nm_tt)
print("Accuracy_test:", accuracy(conf_matr_nm_tt))

Матрица ошибок
[[16  1]
 [ 0 13]]
Accuracy_test: 0.9666666666666667


Accuracy на тренировочных данных при использовании nesterov momentum точно такое же, как и при методе градиентного спуска. Вместе с тем, нельзя не отметить, что при использовании тестовых данных (которые модель еще не видела) результат оказался выше у метода nesterov momentum.

### Шаг 5. Метод rmsprop

Данный метод позволяет также как и нестеров моментум использовать экспоненциально затухающее бегущее среднее, но при этом вместо полной суммы обновлений весов (которые обновляются часто) используется усреднённый по истории квадрат градиента.  
Соответственно, мы лишь немного видоизменим функцию, которую писали для нестеров моментум:

In [54]:
def rmsprop(start, l_rate, n_epochs, train, y):
    coef_rmsp = start
    grad_square = np.zeros(5)
    grad = np.zeros(5)
    rmsp_cost = []
    a = 0.99
    eps = 0.0000001     # добавим сглаживающий параметр, необходимый, чтобы избежать деления на 0
    for i in range(n_epochs):
        sigm = predict(coef_rmsp, train)
        loss = - np.mean(np.log(sigm) * y + np.log(1 - sigm) * (1 - y))
        rmsp_cost.append(loss)
        
        grad = np.matmul((sigm - y), train)/len(sigm)

        grad_square = a * grad_square + (1 - a)  * grad ** 2

        coef_rmsp -= l_rate * grad / np.sqrt(grad_square + eps)
        i += 1
    return coef_rmsp

In [55]:
coef_rmsp = rmsprop(coef, 0.01, 500, f_tr, t_tr)
coef_rmsp

array([-3.77235683, -2.06380094, -2.4192069 ,  3.53262128,  3.82999739])

In [56]:
pred_rmsp_train = []
for row in features_train.to_numpy():
    yhat_rmsp_tr = log_reg(row, coef_rmsp)
    print("Predicted=%.3f [%d]" % (yhat_rmsp_tr, round(yhat_rmsp_tr)))
    pred_rmsp_train.append(round(yhat_rmsp_tr))

Predicted=0.068 [0]
Predicted=0.032 [0]
Predicted=0.969 [1]
Predicted=0.178 [0]
Predicted=0.984 [1]
Predicted=0.192 [0]
Predicted=0.867 [1]
Predicted=0.943 [1]
Predicted=0.087 [0]
Predicted=0.280 [0]
Predicted=0.055 [0]
Predicted=0.370 [0]
Predicted=0.015 [0]
Predicted=0.911 [1]
Predicted=0.336 [0]
Predicted=0.033 [0]
Predicted=0.731 [1]
Predicted=0.023 [0]
Predicted=0.917 [1]
Predicted=0.196 [0]
Predicted=0.021 [0]
Predicted=0.171 [0]
Predicted=0.019 [0]
Predicted=0.042 [0]
Predicted=0.040 [0]
Predicted=0.983 [1]
Predicted=0.116 [0]
Predicted=0.991 [1]
Predicted=0.060 [0]
Predicted=0.833 [1]
Predicted=0.922 [1]
Predicted=0.846 [1]
Predicted=0.989 [1]
Predicted=0.015 [0]
Predicted=0.998 [1]
Predicted=0.962 [1]
Predicted=0.061 [0]
Predicted=0.999 [1]
Predicted=0.923 [1]
Predicted=0.830 [1]
Predicted=0.674 [1]
Predicted=0.118 [0]
Predicted=0.977 [1]
Predicted=0.004 [0]
Predicted=0.957 [1]
Predicted=0.964 [1]
Predicted=0.848 [1]
Predicted=0.020 [0]
Predicted=0.984 [1]
Predicted=0.978 [1]


In [57]:
conf_matr_rmsp_tr = my_confusion_matrix(t_tr, pred_rmsp_train)
print("Матрица ошибок")
print(conf_matr_rmsp_tr)
print("Accuracy_test:", accuracy(conf_matr_rmsp_tr))

Матрица ошибок
[[32  1]
 [ 0 37]]
Accuracy_test: 0.9857142857142858


Тест:

In [58]:
pred_rmsp_test = []
for row in features_test.to_numpy():
    yhat_rmsp_tt = log_reg(row, coef_rmsp)
    print("Predicted=%.3f [%d]" % (yhat_rmsp_tt, round(yhat_rmsp_tt)))
    pred_rmsp_test.append(round(yhat_rmsp_tt))

Predicted=0.553 [1]
Predicted=0.947 [1]
Predicted=0.960 [1]
Predicted=0.034 [0]
Predicted=0.115 [0]
Predicted=0.113 [0]
Predicted=0.558 [1]
Predicted=0.953 [1]
Predicted=0.061 [0]
Predicted=0.018 [0]
Predicted=0.438 [0]
Predicted=0.036 [0]
Predicted=0.710 [1]
Predicted=0.811 [1]
Predicted=0.980 [1]
Predicted=0.123 [0]
Predicted=0.625 [1]
Predicted=0.642 [1]
Predicted=0.029 [0]
Predicted=0.017 [0]
Predicted=0.990 [1]
Predicted=0.608 [1]
Predicted=0.094 [0]
Predicted=0.035 [0]
Predicted=0.873 [1]
Predicted=0.015 [0]
Predicted=0.219 [0]
Predicted=0.893 [1]
Predicted=0.130 [0]
Predicted=0.993 [1]


In [59]:
conf_matr_rmsp_tt = my_confusion_matrix(t_tt, pred_rmsp_test)
print("Матрица ошибок")
print(conf_matr_rmsp_tt)
print("Accuracy_test:", accuracy(conf_matr_rmsp_tt))

Матрица ошибок
[[15  2]
 [ 0 13]]
Accuracy_test: 0.9333333333333333


Данный метод показал наивысший результат при анализе доли правильных ответов на тренировочной выборке. Тем не менее, на тестовой выборке он показал результат идентичный градиентному спуску.

### Шаг 6. Выводы

Таким образом, цель данной работы - оптимизировать самостоятельно реализованную логистическую регрессию - достигнута.  
Работа состояла из ряда задач, которые были взяты за основу плана проекта.  
- Первой задачей  была реализация логистической регрессии. Нами были подобраны рандомным способом несколько вариантов коэффициентов для обучения модели. Самый лучший вариант коэффициентов был оставлен как основа для последующих методов оптимизации. И всё же рандомный метод перебора параметров был ожидаемо не самым лучшим, о чем свидетельствует достаточно низкий (в сравнении с остальными методами) показатель качества accuracy.  
- В этой связи мы приступили к реализации второй задачи - обучить модель логистической регрессии при помощи метода градиентного спуска. Градиентный спуск показал хороший результат, поэтому он также был зафиксирован и взят за образец.  
- Метод "nesterov momentum" (задача №3) показал себя чуть лучше, чем градиентный спуск: при вычислении accuracy для тренировочной выборки результат был одинаков, но при определении доли правильных ответов на тестовой выборке этот метод дал результат лучше.  
- Последней задачей стала реализация метода rmsprop. При оптимизации работы модели данным методом был достигнут наивысший accuracy на обучающей выборке, однако на тестовой результат был аналогичен результату градинтного спуска.

Таким образом, можно подвести итог, что цель работы выполнена, модель обучена, методы реализованы. Лучше всего в данном случае на тестовой выборке показал себя метод nesterov momentum.